### 임상시험 데이터셋 모델
임상시험 데이터셋의 표준인 CDASH, SDTM, ADaM에 대해 알게된 내용을 정리함.

#### CDASH, SDTM
CDASH는 CRF를 통해 획득된 데이터를 0차적으로 정의하고 SDTM으로 기계 변환 가능하게 하는 규칙임. 임상시험용 앱 프론트앤드서 필드에 입력된 데이터가 백앤드에 어떻게 저장되는지를 규정하고, 기정의 디자인 요소를 제공함.

SDTM은 FDA 제출용 데이터셋으로 표준화된 규칙에 의거 변수의 이름과 내용을 정하고 카테고레이즈할 수 있는 프로토콜임.

#### 특징
- CRF에서 생일 필드를 년, 월, 일의 문자열로 텍스트 박스에 입력하면, 각각을 (BRTHYR, BRTHMO, BRTHDY)라는 변수(컬럼)으로 냄
    - SDTM의 BRTHDTC = (BRTHYR + BRTHMO + BRTHDY)으로 파싱함
- CDASH에는 16개 도메인이 있음
    - (AE, CO, CM, DM, DS, DA, EG, EX, IE, LB, MH, PE, DV, SC, SU, VS)
    - 도메인마다 CRF 페이지가 구성되는 것은 아님
- CDASH로 모든 CRF 항목이 커버되지 않으며 CDASH를 벗어날수록 SDTM 변환이 어려워질 것임
    - 이러면 SUPP 데이터셋 사용 필요
- 컬럼 값은 여러 CRF 필드의 파싱 결과이며 필드와 반드시 1:1 대응되지는 않음
- SDTM 데이터셋은 롱 포맷(vertical)임
    - 각 데이터셋에서 인덱스(key)로 사용되는 컬럼이 규정됨
        - {"CM":"STUDYID", "USUBJID", "CMTRT", "CMSTDTC"}
    - 각 데이터셋의 키 컬럼을 인덱스로 했을 때 개수가 레코드 개수임

#### SDTM 비슷하게 바꾸기
[dmisimportant 연습용 데이터셋](cafe.naver.com/dmisimportant/104)의 VS를 SDTM 비슷하게 만들고자 함.

- 필요 모듈 로드, 스펙시트 경로 설정
- 스펙시트를 매퍼(딕셔너리)로서 읽음
    - 예전 노트북에서의 스펙시트 클래스를 딕셔너리 매퍼를 바로 내도록 바꿈
- sas7bdat 로드

In [4]:
import os
from dm import *
specpath="C:/code/CUBEDEMO2017/spec.xlsx"
datapath="C:/code/CUBEDEMO2017/SASSET/"
spec=Spec(pd.read_excel(specpath))

sasobj=[obj for obj in os.scandir(datapath) if any(map(obj.path.lower().__contains__,ext)) and obj.is_file()]
data={os.path.splitext(obj.name)[0].upper():read_sas_(obj.path) for obj in sasobj}
print(ornament,"domain:\n",data.keys(),len(data),"domains")

spec.map

c:\code\pub\ct\dm.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spec["CODE"]=[dict(q.split(":") for q in w.split("|")) if isinstance(w,str) else w for w in spec.CODE]


---------- domain:
 dict_keys(['AE', 'AY', 'CM', 'CT', 'CY', 'DA', 'DM', 'DS', 'DY', 'EF', 'EG', 'EN', 'ES', 'IE', 'IP', 'LB', 'LC', 'LY', 'MH', 'MY', 'PD', 'PG', 'RN', 'SN', 'SU', 'SV', 'VS']) 27 domains


{('EN', 'SUBJID'): {'PGNM': 'EN',
  'PGNO': 1,
  'PAGE_LABEL': 'Enrollment',
  'CRF_LABEL': 'Enrollment',
  'VISIT': '0',
  'ITEM_SEQ': 1,
  'ITEM_LABEL': 'Screening Number',
  'CODE': nan,
  'LAYOUT': 'SYSDEFINED',
  'KEY': 1.0,
  'TYPE_LENGTH': 'C8',
  'VIEW_TYPE': 'nvarchar2(8)'},
 ('EN', 'VERSION'): {'PGNM': 'EN',
  'PGNO': 1,
  'PAGE_LABEL': 'Enrollment',
  'CRF_LABEL': 'Enrollment',
  'VISIT': '0',
  'ITEM_SEQ': 2,
  'ITEM_LABEL': 'Version',
  'CODE': nan,
  'LAYOUT': 'SYSDEFINED',
  'KEY': 1.0,
  'TYPE_LENGTH': 'C10',
  'VIEW_TYPE': 'nvarchar2(10)'},
 ('EN', 'ICDTC'): {'PGNM': 'EN',
  'PGNO': 1,
  'PAGE_LABEL': 'Enrollment',
  'CRF_LABEL': 'Enrollment',
  'VISIT': '0',
  'ITEM_SEQ': 3,
  'ITEM_LABEL': 'Date of informed consent',
  'CODE': nan,
  'LAYOUT': 'DATE',
  'KEY': nan,
  'TYPE_LENGTH': 'YYYY-MM-DD',
  'VIEW_TYPE': 'nvarchar2(10)'},
 ('SV', 'SUBJID'): {'PGNM': 'SV',
  'PGNO': 2,
  'PAGE_LABEL': 'Visit',
  'CRF_LABEL': 'Visit',
  'VISIT': '1,2,3,4,5,6,7,2001,4001',
  'ITEM

- VS를 vs로 선언함

In [5]:
vs=data["VS"]
vs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   SUBJID  344 non-null    object 
 1   VISIT   344 non-null    float64
 2   VSYN    343 non-null    float64
 3   VSDTC   236 non-null    object 
 4   HEIGHT  83 non-null     float64
 5   WEIGHT  235 non-null    float64
 6   SYSBP   269 non-null    float64
 7   DIABP   245 non-null    float64
 8   PULSE   234 non-null    float64
 9   RESP    234 non-null    float64
 10  TEMP    234 non-null    float64
dtypes: float64(9), object(2)
memory usage: 29.7+ KB


- VS의 인덱스(key) 선언
    - vs의 인덱스 설정

In [6]:
key=["SUBJID","VISIT","VSYN"]
vs=data["VS"].set_index(key)
vs

VSDTC  HEIGHT  WEIGHT  SYSBP  DIABP  PULSE  RESP  \
SUBJID   VISIT  VSYN                                                          
S-1Z-005 1.0    1.0   2016-02-10   200.0   115.0  170.0  120.0  120.0  30.0   
S-1Z-010 1.0    NaN          NaN     NaN     NaN  140.0    NaN    NaN   NaN   
         5.0    1.0          NaN     NaN     NaN  139.0   89.0    NaN   NaN   
S-1Z-012 1.0    2.0          NaN     NaN     NaN    NaN    NaN    NaN   NaN   
S-1Z-018 1.0    1.0   2016-11-07   165.0    55.0  145.0   89.0   78.0  20.0   
...                          ...     ...     ...    ...    ...    ...   ...   
S-US-004 2.0    2.0          NaN     NaN     NaN    NaN    NaN    NaN   NaN   
         2001.0 2.0          NaN     NaN     NaN    NaN    NaN    NaN   NaN   
S-US-005 1.0    1.0   2016-03-01   150.0    60.0  150.0   85.0   30.0  30.0   
         2.0    1.0   2016-03-31     NaN   154.0  155.0  100.0   60.0  40.0   
S-US-007 1.0    1.0   2017-03-01   140.0    20.0   20.0  160.0   30.0  25.0   

                      TEMP  
SUBJID   VISIT  VSYN        
S-1Z-005 1.0    1.0   38.0  
S-1Z-010 1.0    NaN    NaN  
         5.0    1.0    NaN  
S-1Z-012 1.0    2.0    NaN  
S-1Z-018 1.0    1.0   36.7  
...                    ...  
S-US-004 2.0    2.0    NaN  
         2001.0 2.0    NaN  
S-US-005 1.0    1.0   37.0  
         2.0    1.0   40.0  
S-US-007 1.0    1.0   37.0  

[344 rows x 8 columns]

- VSDTC는 vs 인덱스에 맞춰지는 값이므로 따로 둠
    - 즉, VSDTC는 SDTM 기반 VS 데이터셋에서 하나의 컬럼으로 됨

In [7]:
vsdtc=vs.VSDTC
vsdtc

SUBJID    VISIT   VSYN
S-1Z-005  1.0     1.0     2016-02-10
S-1Z-010  1.0     NaN            NaN
          5.0     1.0            NaN
S-1Z-012  1.0     2.0            NaN
S-1Z-018  1.0     1.0     2016-11-07
                             ...    
S-US-004  2.0     2.0            NaN
          2001.0  2.0            NaN
S-US-005  1.0     1.0     2016-03-01
          2.0     1.0     2016-03-31
S-US-007  1.0     1.0     2017-03-01
Name: VSDTC, Length: 344, dtype: object

- 이제 vs에서 VSDTC 외 컬럼을 스태킹함
    - 스태킹은 카테고리 값을 가지는 컬럼을 그 값에 따라 인덱스에 할당함
    - 피벗의 고수준 메서드
- 컬럼 이름 바꿈

In [9]:
vs_=vs.iloc[:,vs.columns.get_loc("VSDTC")+1:].stack().reset_index()
vs_.columns=["SUBJID","VISIT","VSYN","VSTESTCD","VSORRES"]
vs_

,SUBJID,VISIT,VSYN,VSTESTCD,VSORRES
0,S-1Z-005,1.0,1.0,HEIGHT,200.0
1,S-1Z-005,1.0,1.0,WEIGHT,115.0
2,S-1Z-005,1.0,1.0,SYSBP,170.0
3,S-1Z-005,1.0,1.0,DIABP,120.0
4,S-1Z-005,1.0,1.0,PULSE,120.0
...,...,...,...,...,...
1529,S-US-007,1.0,1.0,SYSBP,20.0
1530,S-US-007,1.0,1.0,DIABP,160.0
1531,S-US-007,1.0,1.0,PULSE,30.0
1532,S-US-007,1.0,1.0,RESP,25.0


- 남겨둔 vsdtc를 vsdtc의 인덱스이자 아까 선언한 vs의 인덱스를 기준으로 합침
- VSTEST 컬럼을 VSTESTCD의 내용에 따라 스펙시트 매퍼의 라벨 값으로 만들어 줌
- SDTM-like 후처리 종료

In [10]:
vs_.merge(vsdtc,on=key).assign(VSTEST=vs_.VSTESTCD.apply(lambda q:spec.map[("VS",q)]["ITEM_LABEL"].upper()))

,SUBJID,VISIT,VSYN,VSTESTCD,VSORRES,VSDTC,VSTEST
0,S-1Z-005,1.0,1.0,HEIGHT,200.0,2016-02-10,HEIGHT
1,S-1Z-005,1.0,1.0,WEIGHT,115.0,2016-02-10,WEIGHT
2,S-1Z-005,1.0,1.0,SYSBP,170.0,2016-02-10,SYSTOLIC BLOOD PRESSURE
3,S-1Z-005,1.0,1.0,DIABP,120.0,2016-02-10,DIASTOLIC BLOOD PRESSURE
4,S-1Z-005,1.0,1.0,PULSE,120.0,2016-02-10,PULSE RATE
...,...,...,...,...,...,...,...
1529,S-US-007,1.0,1.0,SYSBP,20.0,2017-03-01,SYSTOLIC BLOOD PRESSURE
1530,S-US-007,1.0,1.0,DIABP,160.0,2017-03-01,DIASTOLIC BLOOD PRESSURE
1531,S-US-007,1.0,1.0,PULSE,30.0,2017-03-01,PULSE RATE
1532,S-US-007,1.0,1.0,RESP,25.0,2017-03-01,RESPIRATORY RATE
